In [19]:
import cv2
import torch

pict_list = []
cur_page = 0
while True:
    with open('tt.txt', 'r') as a:
        d = a.readline().replace('\n', '')
        if d!='':
            d = int(d)
        if d == 0:
            break

with open('division_name.txt', 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        pict_list.append(line.replace('\n', ''))

step_parts = [['squid', 'leg', 'doorhole'],
              ['squid'],
              ['wheel'],
              ['stick'],
              ['stick', 'squidhole'],
              ['pump', 'seat', 'seathole'],
              ['stick', 'pump', 'underhole']]

part_color = {'seat':(49, 53, 82),
            'pump':(184, 64, 94),
            'stick':(46, 176, 134),
            'squid':(70, 36, 76),
            'leg':(199, 75, 80),
            'wheel':(107, 203, 119),
            'seathole':(77, 150, 255),
            'underhole':(77, 150, 255),
            'squidhole':(77, 150, 255),
            'doorhole':(77, 150, 255)}

model = torch.hub.load('ultralytics/yolov5', 'custom', './finalmodel2.pt')
model.conf = 0.4

capture = cv2.VideoCapture(0)
w = round(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
h = round(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = capture.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
output_capture = cv2.VideoWriter('output.avi', fourcc, fps, (w, h))

frame_hole_label = []

while cv2.waitKey(33) < 0:
    with open('tt.txt', 'r') as a:
        d = int(a.readline().replace('\n', ''))
        if cur_page != d:
            cur_page = d

    ret, frame = capture.read()
    manual = cv2.imread(pict_list[cur_page])
    px = manual.shape[1]
    py = manual.shape[0]
    if px > py:
        px = px // 4
        py = py // 4
    else:
        px = px // 5
        py = py // 5
    ma = cv2.resize(manual, (px, py), interpolation = cv2.INTER_AREA)
    frame[:py, :px] = ma
    check_hole = 0
    
    frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    results = model(frame)

    # 모델에서 얻어온 results 정리
    df = results.pandas().xyxy[0]
    obj_list = []
    for i in range(len(df)):
        if py > df['ymax'][i] and px > df['xmax'][i]:
            continue

        if df['name'][i] in step_parts[cur_page]:
            obj_xmin = int(df['xmin'][i])
            obj_ymin = int(df['ymin'][i])
            obj_xmax = int(df['xmax'][i])
            obj_ymax = int(df['ymax'][i])
            obj_class = df['class'][i]
            obj_name = df['name'][i]
            obj_dict = {'class' : obj_name,
                        'idx' : obj_class,
                        'bbox' : [obj_xmin, obj_ymin, obj_xmax, obj_ymax]}
            obj_list.append(obj_dict)

            if obj_name[-4:] == 'hole':
                check_hole = 1
                frame_hole_label.append(obj_name + ' ' + str(obj_xmin) + ' ' + str(obj_ymin) + ' ' + str(obj_xmax) + ' ' + str(obj_ymax) + '\n')

    if check_hole == 0:
        frame_hole_label.append('\n')
    
#     # 3D object
#     frame = drawFinal.process(img, obj_list)
    
    for per in obj_list:
        frame = cv2.rectangle(frame, (per['bbox'][0], per['bbox'][1]-30), (per['bbox'][0]+60, per['bbox'][1]), part_color[per['class']], -1)
        frame = cv2.putText(frame, per['class'], (per['bbox'][0]+3, per['bbox'][1]-5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (247, 247, 247), 1)
        frame = cv2.rectangle(frame, (per['bbox'][0], per['bbox'][1]), (per['bbox'][2], per['bbox'][3]), part_color[per['class']], 3)
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    cv2.imshow('glass output', frame)

    output_capture.write(frame)
with open('per_frame_label.txt', 'w') as f:
    for line in frame_hole_label:
        f.write(line)
        
capture.release()
cv2.destroyAllWindows()